# Time-varying loadings

**Exchange rates and macroeconomic fundamentals: Evidence of instabilities from time‐varying factor loadings**

Eric Hillebrand, Jakob Guldbæk Mikkelsen, Lars Spreng, Giovanni Urga

In [ ]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
import numpy as np
from latentfactors import cfg

k = 3

fx = pd.read_excel(fr"{cfg.fldr}/data/hmsu/fx.xlsx", index_col=0)
eco_us = pd.read_excel(fr"{cfg.fldr}/data/hmsu/usa.xlsx", index_col=0)

# Log-difference FX
dlog_fx = fx.apply(np.log).diff().reindex(eco_us.index).mul(-1)
t, n = dlog_fx.shape

## Factors estimation

In [ ]:
# Standardize economic indicators
scaler = StandardScaler()
scaler.fit(eco_us)
X = scaler.transform(eco_us)

# Transform them into actors
np.random.seed(1234)
svd = TruncatedSVD(n_components=k)
svd.fit(X)
V = svd.components_  # Right singular vectors
S = svd.singular_values_

U = X @ V.T @ np.linalg.inv(np.diag(S))  # Left singular values
mF = np.multiply(U, np.sqrt(t))

## Estimate loadings

In [ ]:
mBeta = {}

# For-loop
crncy = "AUD"

# OLS Estimates
r = dlog_fx[crncy].values.reshape(-1, 1)  # FX Series
b = np.linalg.inv(mF.T @ mF) @ mF.T @ r  # OLS Beta
r_hat = mF @ b  # OLS predicted values
e = r - r_hat
s_e = e.T @ e / t  # Asymptotic OLS variance
# TODO: Compute White VCV
# TODO: Compute HAC VCV
L_ols = -1/2 * t * np.log(s_e.item())  # Likelihood values for restricted model with all loadings constant
# mBeta[crncy] = b.flatten()

In [ ]:
def KFNomean(vP, Data, matF):
    """The Kalman filter for DFM by concentrating out means.

    Input:
        vP: vector of parameters, no mean parameters
        Data: Data
        matF: principal components/factors

    Output:
        vF: vector of prediction error variances
        vV: vector of prediction errors for data
        mVf: matrix of prediction errors for factors
        vMean: mean estimate
    """
    pass

In [ ]:
vP0 = np.concatenate([np.zeros((2 * k, 1)), np.log(s_e)])
print(vP0)

LoglikNomean.m (before KF)

In [ ]:
# Parameters
vP = vP0.copy()
Data = r.copy()
matF = mF.copy()

# 
iT = Data.shape[0]
iR = mF.shape[1]
vPtrans = vP.copy()

for i in range(iR):
    vPtrans[2 * i + 1] = np.exp(vP[2 * i + 1])

vPtrans[-1] = np.exp(vP[-1])

print(vPtrans)

In [145]:
def KFNomean(vP, Data, matF):
    """
    The Kalman filter for DFM by concentrating out means

    Input:    vP - n-by-1 numpy vector of parameters, no mean parameters
              Data - x-by-1 numpy vector of observations
              matF - principal components/factors
    Output:   vF - vector of prediction error variances
              vV - vector of prediction errors for data
              mVf - matrix of prediction errors for factors
              vMean - Mean estimate
    """

    # Specify the matrices
    iR = matF.shape[1]
    f = matF
    T = np.eye(iR)
    Q = np.eye(iR)
    P_t = np.eye(iR)
    for j in range(iR):
        T[j, j] = vP[2 * j].item()
        Q[j, j] = vP[2 * j + 1].item()
        P_t[j, j] = Q[j, j].item() / (1 - T[j, j].item() ** 2)

    sigmaeps = vP[-1].item()

    # KF
    iT = len(Data)  # Now determined by the length of the Data vector
    vF = np.zeros(iT + 1)
    vV = np.zeros(iT + 1)
    mVf = np.zeros((iT + 1, iR))
    a_t = np.zeros(iR)  # Estimate mean in measurement eq
    Af_t = np.zeros((iR, iR))

    for i in range(iT):
        vV[i] = Data[i].item() - f[i, :] @ a_t  # Data prediction error
        mVf[i, :] = f[i, :] - f[i, :] @ Af_t  # Factor prediction error
        vF[i] = f[i, :] @ P_t @ f[i, :].T + sigmaeps
        temp = P_t @ f[i, :].T / vF[i]  # Gain
        a_tt = a_t + temp * vV[i]  # Updating equation
        Af_tt = Af_t + temp.reshape(-1, 1) @ mVf[i, :].reshape(1, -1)
        P_tt = P_t - temp.reshape(-1, 1) @ f[i, :].reshape(1, -1) @ P_t  # Updating equation
        a_t = T @ a_tt  # Prediction equation (Parameters)
        Af_t = T @ Af_tt
        P_t = T @ P_tt @ T.T + Q  # Prediction equation (Parameter variance)

    vV = vV[:iT]  # Delete the last element as it is only relevant for forecasting
    mVf = mVf[:iT, :]  # Delete the last element as it is only relevant for forecasting
    vF = vF[:iT]  # Same here
    XdivF = (mVf / vF[:, np.newaxis]).T
    vMean = np.linalg.solve(XdivF @ mVf, XdivF @ vV)  # GLS estimate

    return vV, mVf, vF, vMean

In [146]:
KFNomean(vP=vPtrans.copy(), Data=r.copy(), matF=mF.copy())

(array([-5.59657822e-03,  2.30119228e-02,  1.60276961e-02,  2.12545116e-02,
         2.05834549e-02, -3.16941132e-02, -3.38293161e-02, -2.49670484e-03,
         1.10760785e-02,  4.59028162e-03, -1.44353800e-02,  1.02029076e-02,
        -7.70699215e-03, -1.81020886e-02,  1.61450108e-02,  1.36045308e-02,
         1.53123312e-02, -2.99680861e-03, -7.81393656e-03, -1.95621632e-02,
        -3.07142950e-02,  5.49522586e-03,  8.92286878e-03,  4.58939271e-03,
        -8.20362594e-03, -3.09098105e-04, -1.39500553e-02, -2.79123156e-02,
        -2.66386712e-03, -1.07824189e-02, -3.61687360e-02,  1.02177366e-03,
        -2.50848134e-02,  1.43276658e-02,  3.24358761e-02,  8.91640807e-03,
        -1.79134197e-02, -3.58366990e-02,  4.44859626e-03,  2.83905058e-04,
        -3.96484424e-02,  1.40953229e-02,  5.84332455e-03,  1.33962273e-02,
         3.28450439e-02,  2.85168957e-02, -7.48089848e-03,  7.22316283e-03,
         1.20830725e-02,  1.15936885e-02, -4.76552996e-03,  1.15053221e-02,
         4.7

In [ ]:
from filterpy.kalman import KalmanFilter
